import the require packages

In [1]:
import time
import datetime
import requests
import re
import json
from bs4 import BeautifulSoup 
import pandas as pd
from datetime import datetime
import os
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
import urllib.parse
from urllib.parse import quote, unquote, urljoin, urlparse, parse_qs, urlencode, urlunparse
import csv

In [2]:
## url for searching page
#f'https://gcis.nat.gov.tw/cod/searchAction.do?method=doSearch&keyword=&description=&code={keyword}&codeName=&codeEname=&search=%E6%9F%A5%E8%A9%A2'

In [ ]:
## only use when using colab
# from google.colab import files
# uploaded = files.upload()

get the information directly from the page that redirect from click the button name '細類列表'

In [3]:
def get_info(keyword):
    url = f'https://gcis.nat.gov.tw/cod/browseAction.do?method=browse&layer=4&code={keyword}'

    try:
        res = requests.get(url)
        res.raise_for_status()

        soup = BeautifulSoup(res.text, "html.parser")
        parent_table = soup.find('table', summary="排版表格")

        if parent_table:
            nested_table = parent_table.find('table', summary="營業項目代碼詳細內容，第一欄為營業項目代碼，第二欄為營業項目，第三欄為英文營業項目，第四欄為與行業統計分類對照，第五欄為定義內容")
            
            if nested_table:
                ## use dictionary to save the information get 
                data = {}
                rows = nested_table.find_all('tr')
                
                for row in rows:
                    th = row.find('th')
                    td = row.find('td')
                    
                    if th and td:
                        key = th.text.strip()
                        value = td.text.strip()
                        if key in ["營業項目代碼", "營業項目", "英文營業項目", "與行業統計分類對照", "定義內容"]:
                            data[key] = value
                # print(data)
                return data
            else:
                print('Nested table not found')
        else:
            print('Parent table not found')

    except requests.RequestException as e:
        print(f"Request failed: {e}")
        return None

# Example usage
#get_info('A201010')


get the number part from the result of column name '與行業統計分類對照' and save in a new column name '僅代碼部分' \
and save the context part into new column name '敘述說明'

In [4]:
def process_industry_classification(value):
    pattern = re.compile(r'(\d{4})\t([^\d]+)')
    matches = pattern.findall(value)
    
    processed_lines = []
    codes = []
    descriptions = []

    for code, description in matches:
        description = description.strip()
        processed_lines.append(f"{code}\t{description}")
        codes.append(code)
        descriptions.append(description)
    
    processed_value = '\n'.join(processed_lines)
    extracted_codes = '\n'.join(codes)
    description_text = '\n'.join(descriptions)
    
    return processed_value, extracted_codes, description_text

save the information with the encode utf-8 with BOM to make sure it can be opened by the excel

In [5]:
def save_to_csv_if_match(data, keyword, output_file):
    fieldnames = ['營業項目代碼', '營業項目', '與行業統計分類對照', '僅代碼部分', '敘述說明', '定義內容']
    file_exists = os.path.isfile(output_file)

    with open(output_file, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        if not file_exists:
            csvfile.write('\ufeff')
            writer.writeheader()
        
        if data:
            if data.get('營業項目代碼') == keyword:
                processed_value, codes, descriptions = process_industry_classification(data.get('與行業統計分類對照', ''))
                
                writer.writerow({
                    '營業項目代碼': data.get('營業項目代碼', ''),
                    '營業項目': data.get('營業項目', ''),
                    '與行業統計分類對照': processed_value,
                    '僅代碼部分': codes,
                    '敘述說明': descriptions,
                    '定義內容': data.get('定義內容', '')
                })
                print(data)
        else:
            # Write only the keyword with other fields empty
            writer.writerow({
                '營業項目代碼': keyword,
                '營業項目': '',
                '與行業統計分類對照': '',
                '僅代碼部分': '',
                '敘述說明': '',
                '定義內容': ''
            })

    print(f'Data saved to {output_file}')


read the code list in the .txt file and search for the result

In [6]:
if __name__ == "__main__":
    with open('code_list.txt', 'r') as f:
        lines = f.readlines()
        for keyword in lines:
            keyword = keyword.strip() 
            print(keyword)
            data = get_info(keyword)
            print(data)
            save_to_csv_if_match(data, keyword, 'result0916.csv')

A101011
{'營業項目代碼': 'A101011', '營業項目': '種苗業', '與行業統計分類對照': '0119\t其他農作物栽培業', '定義內容': '依植物品種及種苗法規定，從事繁殖、輸出入、銷售種苗之事業者。'}
{'營業項目代碼': 'A101011', '營業項目': '種苗業', '與行業統計分類對照': '0119\t其他農作物栽培業', '定義內容': '依植物品種及種苗法規定，從事繁殖、輸出入、銷售種苗之事業者。'}
Data saved to result0916.csv
A101020
{'營業項目代碼': 'A101020', '營業項目': '農作物栽培業', '與行業統計分類對照': '0111\t稻作栽培業\r0112\t雜糧栽培業\r0113\t特用作物栽培業\r0114\t蔬菜栽培業\r0115\t果樹栽培業', '定義內容': '從事稻米、蔬菜、果樹、甘蔗及雜糧等作物栽培之行業。包括稻作栽培業、雜糧栽培業、蔬菜栽培業、果樹栽培業、甘蔗栽培業。'}
{'營業項目代碼': 'A101020', '營業項目': '農作物栽培業', '與行業統計分類對照': '0111\t稻作栽培業\r0112\t雜糧栽培業\r0113\t特用作物栽培業\r0114\t蔬菜栽培業\r0115\t果樹栽培業', '定義內容': '從事稻米、蔬菜、果樹、甘蔗及雜糧等作物栽培之行業。包括稻作栽培業、雜糧栽培業、蔬菜栽培業、果樹栽培業、甘蔗栽培業。'}
Data saved to result0916.csv
A101030
{'營業項目代碼': 'A101030', '營業項目': '特用作物栽培業', '與行業統計分類對照': '0113\t特用作物栽培業', '定義內容': '從事纖維料、油料、糖料、嗜好料、香料、藥料及工業原料等特用作物栽培之行業。如棕櫚栽培、苧麻栽培、亞麻栽培、大甲藺栽培、芏苡（三角藺）栽培、向日葵栽培、油菜籽栽培、葛鬱金（粉薯）栽培、甜菜栽培、茶葉栽培、菸草栽培、胡椒栽培、花椒栽培、香茅草栽培、芥末籽栽培、杭菊栽培、除蟲菊栽培、枸杞栽培、黃蓍栽培、麥門冬栽培、桑樹栽培、 棉花栽培、瓊麻栽培、黃麻栽培、洋麻（鐘麻）栽培、芝麻栽培、蓖麻籽栽培、樹薯栽培、甜菊栽培、咖啡栽培、可可豆栽培、蛇麻栽培、茴香栽培、仙草栽培、洛神葵栽